# Test swaps

In [ ]:
from sugar import AsyncOPChainSimnet, OPChainSimnet, AsyncOPChain, AsyncBaseChain, AsyncLiskChain
from sugar.superswap import MockSuperswapRelayer, AsyncSuperswap, Superswap
from sugar.helpers import require_supersim, atime_it
from fastcore.test import test_eq
from typing import List, Dict, Any, Optional

require_supersim()

In [ ]:
# velo -> usdc
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(from_token=op.velo, to_token=op.usdc, amount=10)

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))

    with OPChainSimnet() as op_simnet_sync:
        print(op_simnet_sync.swap_from_quote(quote, slippage=0.1))

# velo -> eth
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(op.velo, op.eth, 10)

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))

# eth -> velo
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(op.eth, op.velo, 0.001)

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))


Get quote took 3.2378 seconds
AttributeDict({'type': 2, 'status': 1, 'cumulativeGasUsed': 264397, 'logs': [AttributeDict({'address': '0xc38464250F51123078BBd7eA574E185F6623d037', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x000000000000000000000000230c121f55538a05448a9eeb8a510e7e0af39ee9'), HexBytes('0x00000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763c')], 'data': HexBytes('0x0000000000000000000000000000000000000000000000009692b674420d210f'), 'blockHash': HexBytes('0xa357d2212141cbc7da4ac66132a1bd5a6a2597eb3e08a5a4f71e65882550fc4c'), 'blockNumber': 137770186, 'blockTimestamp': '0x6860436c', 'transactionHash': HexBytes('0x93cf51e8f4b68e9c413674ee1aeabb9860398ba12607903da99cbf28de67aebb'), 'transactionIndex': 0, 'logIndex': 0, 'removed': False}), AttributeDict({'address': '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'), HexBytes('

## Superswaps

In [ ]:
# try unsupported chains

async with AsyncOPChainSimnet() as op_sim:
    from_token, to_token = AsyncOPChain.velo, AsyncBaseChain.aero 
    error = None
    try:
        tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=MockSuperswapRelayer()).swap(from_token, to_token, amount=20)
    except ValueError as e:
        error = e
    test_eq(str(error), "Superswap only supports ['OP', 'Lisk', 'Uni']. Got OP -> Base")

# sync version
with OPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.lsk
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=10, slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# async version
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.lsk
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=10, slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# to o_usdt variation
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.o_usdt
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=10, slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 0)

# from o_usdt variation
with OPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.lsk
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=1, slippage=0.1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# async bridge
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.o_usdt
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 0)

# sync bridge
with OPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.o_usdt
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 0)


>>>>>>>>>>>>> b_a: 450054
>>>>>>>>>>>>> d_q: Quote(input=QuoteInput(from_token=Token(chain_id='1135', chain_name='Lisk', token_address='0x1217BfE6c773EEC6cc4A38b5Dc45B92292B6E189', symbol='oUSDT', decimals=6, listed=True, wrapped_token_address=None), to_token=Token(chain_id='1135', chain_name='Lisk', token_address='0xac485391EB2d7D88253a7F1eF18C37f4242D1A24', symbol='LSK', decimals=18, listed=True, wrapped_token_address=None), path=[(LiquidityPoolForSwap(chain_id='1135', chain_name='Lisk', lp='0xDaC26c3ffe00ad0656f8Cd8752186d2f7D9ae789', type=100, token0_address='0x1217BfE6c773EEC6cc4A38b5Dc45B92292B6E189', token1_address='0x4200000000000000000000000000000000000006'), False), (LiquidityPoolForSwap(chain_id='1135', chain_name='Lisk', lp='0xa913882766Af5fFD34E72Bdd646e8E8957Fe1842', type=200, token0_address='0x4200000000000000000000000000000000000006', token1_address='0xac485391EB2d7D88253a7F1eF18C37f4242D1A24'), False)], amount_in=450054), amount_out=1216966971835275403)
>>>>>>>>>> gonn